In [1]:
import pandas as pd

teach_columns = {
    "TIME_CSMTC_SEG E.18-22시" : "저녁 시간에 화장품을 구매하는 고객",
    'INCOME 01_중하계층' : "저소득 고객",
    'INCOME 02_중상계층' : "고소득 고객",
    'ONLINE_CSMTIC_RATE 상' : "온라인으로 화장품을 많이 구매한 고객",
    'DRGSTR_RATE 상' : "drugstore 화장품을 많이 구매한 고객",
    'TIME_SEG 02.오전' : "주요 소비가 오전 시간대인 고객",
    "TIME_SEG 05.야간" : "주요 소비가 야간 시간대인 고객",
    'TIME_CAFE_SEG B.06-10시' : "아침 시간에 카페를 이용 빈도가 높은 고객",
    'TIME_CAFE_SEG B.10-14시' : "점심 시간에 카페를 이용 빈도가 높은 고객",
    'TIME_CAFE_SEG C.14-18시' : "오후 시간에 카페 이용 빈도가 높은 고객",
    'PER_ERADP 상' : "얼리어답터 성향이 있는 고객",
    'LFSTG 01.미혼추정' : "미혼으로 추정되는 고객",
    'PER_DOM_TRV 상' : "국내 여행에 관심이 많은 고객",
    'PER_FRN_TRV 상' : "해외 여행에 관심이 많은 고객",
    'PER_ONLINE 상' : "온라인 구매를 많이 하는 고객",
    'ONLINE_RATE 상' : "온라인 구매가 많은 고객",
    'SKINCARE_CNT_6M 상' : "스킨케어 제품을 선호하는 고객",
    'SKINCARE_CNT_1Y 상' : "스킨케어 제품을 선호하는 고객",
    'PER_PRC_SNSV 상' : "가격에 민감한 고객",
    'PER_INDR_SPRTS 상' : "실내 운동을 많이 하는 고객",
    'PER_OUTDOOR 상' : "외부 활동을 많이 하는 고객",
    'PER_DRIVE 상' : "운전을 많이 하는 고객",
    'PER_REMM 상' : "명품을 많이 구매하는 고객",
    'PER_ONGAME 상' : "온라인 게임을 많이 하는 고객",
    'PER_MDLV 상' : "배달 식품을 자주 먹는 고객",
    'PER_BENEFIT 상' : "혜택에 반응하는 고객",
    'PER_OTT 상' : "OTT 서비스에 관심이 많은 고객",
    'JOB 02_회사원' : "회사원 고객",
    "JOB 01_전문직" : "전문직 고객",
    'PER_PRMM_FASHION 상' : "명품 의류를 많이 구매하는 고객",
    'LFSTG 05.성인자녀추정' : "성인 자녀를 둔 고객",
    'PER_CAFE 상' : "카페를 많이 이용하는 고객",
    "PER_HLTHY_FOOD 상" : "건강식을 많이 구매하는 고객",
    "PER_ORGNIC_FOOD 상" : "유기농 제품을 많이 구매하는 고객",
    "NUTRI_CNT" : "건강 보조 식품을 많이 구매하는 고객",
    "NUTRI_AMT" : "건강 보조 식품을 많이 구매하는 고객",
}

desc = pd.read_csv("description.csv")
desc = desc.dropna(subset = "BASICDESC").reset_index(drop = True)
desc = desc.drop_duplicates(subset = "PRDNM").reset_index(drop = True)
desc = dict(zip(desc["SAPPRDCD"], desc["BASICDESC"]))
# desc

In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from collections import Counter
import ast

warnings.filterwarnings('ignore')
tqdm.pandas()

import requests
import base64
import json

# Azure GPT API 설정
GPT4V_ENDPOINT = "https://apeus-cst-openai.openai.azure.com/openai/deployments/gpt4o/chat/completions?api-version=2024-02-15-preview"
GPT4V_KEY = "fb73c5cdb51d4167b9eb41345d059bfe"

# 헤더 설정
headers = {
    "Content-Type": "application/json;",
    "api-key": GPT4V_KEY,
}

def respond(payload):
    response = requests.post(GPT4V_ENDPOINT, headers=headers, data=json.dumps(payload))
    
    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
        
    else:
        return f"Error {response.status_code}: {response.text}"

In [3]:
df = pd.read_csv("merged_data.csv", encoding = "utf-8-sig", index_col = "Unnamed: 0")
df["sequence"] = df["sequence"].apply(lambda x : ast.literal_eval(x))
df["names"] = df["names"].apply(lambda x : ast.literal_eval(x))
df["tfidf"] = df["tfidf"].apply(lambda x : ast.literal_eval(x))
df["tfidf_name"] = df["tfidf_name"].apply(lambda x : ast.literal_eval(x))
df["external_cluster"] = df["external_cluster"].apply(lambda x : ast.literal_eval(x))

# cf = pd.read_csv("internal_only.csv", encoding = "utf-8-sig", index_col = "Unnamed: 0")
# cf["sequence"] = cf["sequence"].apply(lambda x : ast.literal_eval(x))
# cf["names"] = cf["names"].apply(lambda x : ast.literal_eval(x))
# cf["tfidf"] = cf["tfidf"].apply(lambda x : ast.literal_eval(x))
# cf["tfidf_name"] = cf["tfidf_name"].apply(lambda x : ast.literal_eval(x))

In [5]:
review = pd.read_csv("item2.csv")
review = review.drop_duplicates(subset = "RPRS_PRD_CD").reset_index(drop = True)
review = dict(zip(review["RPRS_PRD_CD"], review["RPRS_PRD_NM"]))

def id_to_name(key):
    try:
        return review[int(float(key))]
    except:
        return key

names = []
for i in df["sequence"]:
    names.append([id_to_name(w) for w in i])
df["names"] = names

tfidf = []
for i in df["tfidf"]:
    tfidf.append([id_to_name(w) for w in i])
df["tfidf_name"] = tfidf
df

,customer_id,cluster,sequence,names,tfidf,tfidf_name,seg_key,external_cluster,external_cluster_num
0,699cb2f62f368a4bcb8faf4cec63d6592f6ef5e48cbd60...,6-0,"[111772235, 111774223, 111770411, 111501723, 1...","[이니스프리 노세범미네랄팩트, 이니스프리 애플씨드클렌징크림, 이니스프리 수퍼화산송이...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6211278,"(PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상)",0.0
1,d5ecc2bf8426946958f568e86f6f17adbec6540a308959...,6-0,"[131173630, 111791677, 111770035, 111770035, 1...","[이니스프리 트루히알루론수분자차선크림, 111791677, 이니스프리 퍼펙트9리페어...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6320663,"(PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상)",0.0
2,662f112160158cbb9da36887f24a2581696ac78085a82f...,6-0,"[111500549, 111705219, 111705312, 131174322, 1...","[이니스프리 올리브리얼로션, 이니스프리 올리브리얼스킨, 이니스프리 그린티밸런싱세트,...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6330067,"(PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상)",0.0
3,2a9a580f05e866ae2f4127342ee6905590460e0c7048db...,6-0,"[111506419, 131174321, 111700874, 131172881, 1...","[111506419, 이니스프리 그린티히알루론산스킨, 이니스프리 그린티클렌징폼, 1...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6220407,"(PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상)",0.0
4,e314e8d74ebbd0458995b3536320d9794354831db4c0b0...,6-0,"[111770033, 111770034, 111772219, 111770031, 1...","[이니스프리 퍼펙트9리페어세럼, 이니스프리 퍼펙트9리페어크림, 이니스프리 에코납작브...","[131172880, 131174283, 131172879, 131171020, 1...","[131172880, 이니스프리 시그니처핸드크림, 131172879, 이니스프리 제...",6310637,"(PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상)",0.0
...,...,...,...,...,...,...,...,...,...
11532,f095e9c4711dab0274daf02673ac46687fa3379414bcb6...,1-0,[111774223],[이니스프리 애플씨드클렌징크림],"[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1211465,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11533,1d18c7072b56816852b3e40da62c42a8eb9bb69da253e5...,1-0,[111771793],[이니스프리 리얼컬러네일],"[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1211248,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11534,3a2f98527824f0e2d5b83d980351f1225180fb2871987d...,1-0,"[111705377, 111771038, 111772219]","[이니스프리 화산송이모공폼, 이니스프리 꽃송이버섯바이탈로션, 이니스프리 에코납작브로...","[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1220492,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0
11535,1c48584732a5775a226b8d687433f6136cbf1d7d6a8779...,1-0,"[131175032, 111770997, 131173960]","[이니스프리 레티놀그린티피디알엔앰플, 이니스프리 잇츠리얼스퀴즈마스크, 이니스프리 비...","[111770034, 111705219, 131172564, 131173332, 1...","[이니스프리 퍼펙트9리페어크림, 이니스프리 올리브리얼스킨, 이니스프리 블랙티유스인핸...",1120803,"(LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상)",5.0


In [6]:
# clu = df.groupby(["cluster", "external_cluster_num"])[["tfidf_name", "external_cluster"]].first()
# clu.index.names = ["internal_cluster_num", "external_cluster_num"]
# clu["cluster"] = range(len(clu["tfidf_name"]))

# change = []
# for i in clu["tfidf_name"]:
#     temp = []
#     for w in i:
#         temp.append(w.replace("IF.", "").replace("이니스프리", "").strip())
#     change.append(temp)
# clu["tfidf_name"] = change

clu = df.groupby(["cluster", "external_cluster_num"])[["tfidf", "external_cluster"]].first()
clu.index.names = ["internal_cluster_num", "external_cluster_num"]
clu["cluster"] = range(len(clu["tfidf"]))

# clu2 = cf.groupby("cluster")["tfidf"].first()
# clu2.index.names = ["internal_cluster_num"]

In [8]:
# description 새로 받기
retrieve = {}
description = {}
for num in tqdm(df.index):
    temp1 = df["tfidf"].loc[num]
    temp2 = df["tfidf_name"].loc[num]
    for a, i in enumerate(temp1):
        retrieve[i] = str(temp2[a]).replace("IF.", "").replace("이니스프리", "").strip()
        try:
            description[retrieve[i]] = desc[i]
        except:
            pass

100%|██████████| 11537/11537 [00:00<00:00, 25464.14it/s]


In [15]:
# 소구점 찾기 (internal only)
result = []
internal_info = []

idx = clu.index.get_level_values("internal_cluster_num").unique()

for temp in tqdm(idx):

    system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
    user1 = """
    내가 제공하는 고객 정보를 활용해서, 이 고객이 어떤 특징을 가지고 있는지 한 문장으로 정의하고, 어떤 정보를 보고 그렇게 정의했는지 그 번호를 알려줘.
    정보는 1개 - 3개 활용해 줘. 각 제품이 어떤 제품인지 알기 위해서 '제품 특징'을 참고해.
    단순히 사용한 정보들을 나열하지 말고, 정보들을 유기적으로 결합해서 고객 특징을 만들어야 해.
    
    우리가 원하는 답변의 예시는 다음과 같아: '바쁜 일상 속에서도 건강한 일상을 살기 위해 이른 아침부터 자기관리와 건강한 루틴을 실천하는 고객. 1번, 3번 정보.'
    """
    assistant1 = "알겠습니다. 정보와 제품 특징을 제공해 주세요."
    
    internal = ""
    tp = list(clu["tfidf"].loc[temp])[0]
    internal_info.append(tp)
    for a, i in enumerate(tp):
        try:
            internal += f"{a}번 정보: 이 고객은 과거에 {retrieve[i]}를 구매한 적이 있어.\n제품 특징: {description[retrieve[i]]}\n"
        except:
            internal += f"{a}번 정보: 이 고객은 과거에 {retrieve[i]}를 구매한 적이 있어.\n"

    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": internal}]}

    decision = True
    while decision:
        try:
            result.append(respond(payload))
            decision = False
        except:
            pass

regexp = re.compile("[0-9].+번 정보")
items = []
for a, i in enumerate(result):
    temp = regexp.findall(i)[0].split("번")
    temp2 = internal_info[a]
    item = []
    for w in temp[:-1]:
        item.append(temp2[int(w[-1])])
    items.append(item)

internal_df = pd.DataFrame({"internal_cluster" : list(idx), "focus_item" : items, "characteristics" : result})
internal_df

100%|██████████| 32/32 [00:56<00:00,  1.77s/it]


,internal_cluster,focus_item,characteristics
0,0-0,"[131170000, 131171318, 131173347]","눈매와 모공 관리에 신경을 쓰면서도, 여름철 방수 메이크업에 특별한 관심을 가진 고..."
1,1-0,"[131172564, 131173041]","지속적인 자기 관리를 중요하게 여기며, 주름과 같이 눈에 띄는 피부 노화 징후를 개..."
2,10-0,"[131174965, 131172564]","지구 환경을 생각하면서도 자신의 피부 건강과 외모를 꼼꼼히 관리하는 고객. 0번, ..."
3,10-2,"[111770997, 111506404]","자연 원료와 순면 제품을 선호하며, 피부와 자신을 소중하게 관리하는 고객. 0번, ..."
4,11-0,"[131172879, 131174282]",자연 친화적인 성분과 제품을 선호하며 자신만의 고유한 향기와 스타일을 중요시하는 고...
5,11-1,"[111770997, 111506404, 111506405]","이 고객은 다양한 피부 고민을 해결하고 자주 화장솜 등으로 피부를 관리하며, 철저한..."
6,12-0,"[131174283, 131171020, 131173425]","피부를 건강하고 자연스럽게 유지하려는 신념을 가진 고객으로, 피부의 생기와 촉촉함을..."
7,14-0,"[131172563, 131170641, 131175032]",자신만의 피부 관리와 세밀한 메이크업 루틴을 통해 완벽한 외모를 유지하려는 디테일한...
8,15-2,"[131171021, 131173502, 131172564]","자연스럽고 건강한 피부 관리를 중요하게 생각하며, 자외선 차단과 노화 방지를 위해 ..."
9,16-0,"[131173707, 131171318, 131174577]","이 고객은 건강한 피부를 유지하며 자연스러운 메이크업을 선호하는 동시에, 자외선과 ..."


In [19]:
# # 소구점 찾기 (internal + external)
# result = []
# internal_info = []
# external_info = []

# for idx, temp in tqdm(enumerate(clu["tfidf"])):

#     system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
#     user1 = """
#     내가 제공하는 고객 정보를 활용해서, 이 고객이 어떤 특징을 가지고 있는지 한 문장으로 정의하고, 어떤 정보를 보고 그렇게 정의했는지 그 번호를 알려줘.
#     내부 정보는 1개 - 3개 활용할 수 있어. 각 제품이 어떤 제품인지 알기 위해서 '제품 특징'을 참고해.
#     외부 정보는 1개 - 3개 활용할 수 있어. 총 활용할 수 있는 정보에는 제한이 없어.
#     단순히 사용한 정보들을 나열하지 말고, 정보들을 유기적으로 결합해서 고객 특징을 만들어야 해.
#     우리가 원하는 답변의 예시는 다음과 같아:
#     '바쁜 일상 속에서도 건강한 일상을 살기 위해 이른 아침부터 자기관리와 건강한 루틴을 실천하는 고객. 내부 1번, 내부 3번, 외부 2번 정보.'
#     '주로 온라인에 몰입되는 도파민 넘치는 생활습관을 가지고 있는 고객. 내부 3번, 외부 0번, 외부 2번 정보.
#     '이른 아침부터 하루를 시작하지만, 늘 시간과 에너지가 부족해 외식이나 차려먹는 밥 대신 배달음식을 시켜먹는 고객. 내부 0번 정보'
#     """
#     assistant1 = "알겠습니다. 정보와 제품 특징을 제공해 주세요."
#     user2 = """
#     외부 0번 정보: 이 고객은 회사원이야.
#     외부 1번 정보: 이 고객은 카페에 지출하는 비용이 많아.
#     외부 2번 정보: 이 고객은 6시 - 12시 사이에 카페를 주로 사용해.
#     외부 3번 정보: 이 고객은 건강식에 대한 지출이 많아.
#     내부 0번 정보: 이 고객은 과거에 아이페놀썬크림을 구매한 적이 있어.
#     내부 1번 정보: 이 고객은 과거에 그린티영양크림을 구매한 적이 있어.
#     """
#     assistant2 = "이른 아침부터 바쁘고 스트레스가 많은 일상을 살며 카페인이 있어야만 사는 고객. 내부 1번, 외부 0번, 외부 2번, 외부 3번 정보."
    
#     internal = ""
#     internal_info.append(temp)
#     for a, i in enumerate(temp):
#         try:
#             internal += f"내부 {a}번 정보: 이 고객은 과거에 {retrieve[i]}를 구매한 적이 있어.\n제품 특징: {description[retrieve[i]]}\n"
#         except:
#             internal += f"내부 {a}번 정보: 이 고객은 과거에 {retrieve[i]}를 구매한 적이 있어.\n"

#     external = ""
#     tp = clu["external_cluster"].iloc[idx]
#     external_info.append(tp)
#     for a, i in enumerate(tp):
#         external += f"외부 {a}번 정보: 이 고객은 {teach_columns[i]}이야.\n"
    
#     payload = {
#         "messages": [
#             {"role": "system", "content": system1},
#             {"role": "user", "content": user1},
#             {"role": "assistant", "content": assistant1},
#             {"role": "user", "content": user2},
#             {"role": "assistant", "content": assistant2},
#             {"role": "user", "content": external + internal}]}

#     decision = True
#     while decision:
#         try:
#             result.append(respond(payload))
#             decision = False
#         except:
#             pass

# internal_regexp = re.compile("내부 [0-9]+번")
# external_regexp = re.compile("외부 [0-9]+번")

# items1 = []
# items2 = []

# for a, i in enumerate(result):
#     temp1 = internal_regexp.findall(i)
#     temp1 = [w[3:-1] for w in temp1]
    
#     temp2 = external_regexp.findall(i)
#     temp2 = [w[3:-1] for w in temp2]

#     check1 = internal_info[a]
#     check2 = external_info[a]

#     item1 = []
#     for w in temp1:
#         item1.append(check1[int(w)])

#     item2 = []
#     for w in temp2:
#         item2.append(check2[int(w)])

#     items1.append(item1)
#     items2.append(item2)
    
# external_df = pd.DataFrame({"internal_item" : items1, "external_item" : items2, "characteristics" : result}, index = clu.index)
# external_df

97it [02:55,  1.81s/it]


In [25]:
# 소구점 찾기 (external only)
result = []
external_info = []

for idx, temp in tqdm(enumerate(clu["tfidf"])):

    system1 = "너는 화장품 기업 소속의 마케팅 매니저야."
    user1 = """
    내가 제공하는 고객 정보를 활용해서, 이 고객이 어떤 특징을 가지고 있는지 한 문장으로 정의하고, 어떤 정보를 보고 그렇게 정의했는지 그 번호를 알려줘.
    정보는 1개 - 3개 활용할 수 있어. 단순히 사용한 정보들을 나열하지 말고, 정보들을 유기적으로 결합해서 고객 특징을 만들어야 해.
    우리가 원하는 답변의 예시는 다음과 같아:
    '바쁜 일상 속에서도 건강한 일상을 살기 위해 이른 아침부터 자기관리와 건강한 루틴을 실천하는 고객. 2번 정보.'
    '주로 온라인에 몰입되는 도파민 넘치는 생활습관을 가지고 있는 고객. 0번, 2번 정보.
    '이른 아침부터 하루를 시작하지만, 늘 시간과 에너지가 부족해 외식이나 차려먹는 밥 대신 배달음식을 시켜먹는 고객. 0번, 1번, 3번 정보'
    """
    assistant1 = "알겠습니다. 정보와 제품 특징을 제공해 주세요."
    
    user2 = """
    0번 정보: 이 고객은 회사원이야.
    1번 정보: 이 고객은 카페에 지출하는 비용이 많아.
    2번 정보: 이 고객은 6시 - 12시 사이에 카페를 주로 사용해.
    3번 정보: 이 고객은 건강식에 대한 지출이 많아.
    """
    assistant2 = "이른 아침부터 바쁘고 스트레스가 많은 일상을 살며 카페인이 있어야만 사는 고객. 0번, 2번, 3번 정보."

    user3 = """
    0번 정보: 이 고객은 OTT 사용 시간이 많아.
    1번 정보: 이 고객은 온라인 쇼핑에 지출하는 비용이 많아.
    2번 정보: 이 고객은 최신 트렌드 패션에 대한 관심이 높아.
    3번 정보: 이 고객은 외부 활동을 많이 해.
    """
    assistant3 = "늘 새로운 트렌드를 쫒고, 전자기기기가 새로 나오면 먼저 써봐야 하고, OTT로 문화생활 섭렵하기 바쁘고 누구보다 발빠르게 소비하는 고객. 0번, 1번, 2번 정보."

    external = ""
    tp = clu["external_cluster"].iloc[idx]
    external_info.append(tp)
    for a, i in enumerate(tp):
        external += f"{a}번 정보: 이 고객은 {teach_columns[i]}이야.\n"
    
    payload = {
        "messages": [
            {"role": "system", "content": system1},
            {"role": "user", "content": user1},
            {"role": "assistant", "content": assistant1},
            {"role": "user", "content": user2},
            {"role": "assistant", "content": assistant2},
            {"role": "user", "content": external}]}

    decision = True
    while decision:
        try:
            result.append(respond(payload))
            decision = False
        except:
            pass

items = []
for a, i in enumerate(result):
    temp = regexp.findall(i)[0].split("번")
    temp2 = external_info[a]
    item = []
    for w in temp[:-1]:
        item.append(temp2[int(w[-1])])
    items.append(item)
    
external_df = pd.DataFrame({"external_item" : items, "characteristics" : result}, index = clu.index)
external_df

97it [02:50,  1.75s/it]


external_item  \
internal_cluster_num external_cluster_num                                                   
0-0                  0.0                                          [PER_OTT 상, PER_MDLV 상]   
                     1.0                   [PER_FRN_TRV 상, PER_DOM_TRV 상, PER_PRC_SNSV 상]   
                     2.0                      [JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]   
1-0                  0.0                          [PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상]   
                     1.0                                  [PER_DOM_TRV 상, PER_PRC_SNSV 상]   
...                                                                                   ...   
7-1                  4.0                                   [PER_FRN_TRV 상, PER_DOM_TRV 상]   
                     5.0                        [LFSTG 05.성인자녀추정, JOB 02_회사원, PER_MDLV 상]   
8-0                  2.0                      [JOB 02_회사원, INCOME 01_중하계층, LFSTG 01.미혼추정]   
9-0                  0.0                          [PER_OTT 상, PER_MDLV 상, PER_PRC_SNSV 상]   
                     1.0                                  [PER_DOM_TRV 상, PER_PRC_SNSV 상]   

                                                                             characteristics  
internal_cluster_num external_cluster_num                                                     
0-0                  0.0                   시간과 에너지가 부족해 편리한 배달 음식을 주로 소비하지만, OTT 서비스를 통해 ...  
                     1.0                   해외 여행에도 많은 관심을 가지고 있지만, 가격에 민감하여 합리적인 가격의 국내 여...  
                     2.0                   금전적 여유는 많지 않지만, 바쁜 회사 생활과 혼자만의 시간을 보내는 데 집중하는 ...  
1-0                  0.0                   가격에 민감하지만, 편리함을 추구하며 배달 식품을 자주 먹으며 OTT 서비스를 즐기...  
                     1.0                   가성비를 중요하게 생각하며 주로 국내 여행에 관심을 두고 있는 고객. 1번, 2번 정보.  
...                                                                                      ...  
7-1                  4.0                   다양한 문화 체험과 여가 활동을 즐기며 국내외 여행에 많은 관심을 갖고 있는 고객....  
                     5.0                   바쁜 직장 생활과 자녀 돌봄의 이중 부담 속에서 편리함을 중시하여 배달 식품을 자주...  
8-0                  2.0                   높은 생활비와 시간적 여유가 부족한 상황에서도 자기계발을 위해 열심히 노력하는 고객...  
9-0                  0.0                   자신만의 편안한 공간에서 여유롭게 OTT 서비스를 즐기기 위해 배달 음식을 선택하고...  
                     1.0                   가격 민감한 성향 때문에 저렴한 비용으로 즐길 수 있는 국내 여행에 주로 관심이 많...  

[97 rows x 2 columns]

In [27]:
internal_df.to_csv("internal_characteristics.csv", encoding = "utf-8-sig")
external_df.to_csv("external_characteristics.csv", encoding = "utf-8-sig")